In [77]:
from datetime import datetime
from datetime import date
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd

In [78]:
sg = Subgrounds()
snapshot = sg.load_api('https://hub.snapshot.org/graphql')

Get the list of spaces. We don't know how many spaces we have, so we have to cycle and only stop when the list is not filled with 1000 rows

In [79]:
spaces=[]
spaces = snapshot.Query.spaces(
  orderBy='id',
  orderDirection='desc',
  first=1000,
)
spaces_list = sg.query_df([
    spaces.id
    ]
  )


In [80]:
spaces2 = []
n=0
t=2
while n<1:
  spaces2 = snapshot.Query.spaces(
    orderBy='created',
    orderDirection='desc',
    first=1000,
    skip = 1000*t
  )
  spaces_list2 = sg.query_df([
    spaces2.id
    ]
  )
  spaces_list = pd.concat([spaces_list,spaces_list2])
  t=t+1
  l=len(spaces_list2)
  print(t,l)
  if len(spaces_list2)<1000: n=1

olympus_coord=spaces_list.index[spaces_list['spaces_id'] == 'olympusdao.eth'].tolist()


3 1000
4 1000
5 1000
6 1000
7 1000
8 1000
9 981


In [81]:
numspaces = len(spaces_list)
numspaces

7981

We get the list of elections  for each snapshot. Note the choice of vote only carries the ID not the question

In [82]:
space_name = spaces_list.iat[1,0]
proposals = snapshot.Query.proposals(
  orderBy='created',
  orderDirection='desc',
  first=1000,
  where=[
    snapshot.Proposal.space == space_name,
    snapshot.Proposal.state == 'closed'
    ##snapshot.Proposal.title == 'OIP-18: Reward rate framework and reduction',
  ]
)

proposals_snapshots = sg.query_df([
    proposals.title,
    proposals.id,
    proposals.body,
    proposals.scores,
    proposals.scores_total
])

proposals_snapshots.insert(0,'dao',space_name)

proposals_snapshots

,dao,proposals_title,proposals_id,proposals_body,proposals_scores,proposals_scores_total
0,🍯dao.eth,Should I get 2 times one ETH,QmQ1s8FHhX2FtFGRcfbbdJmT1sU8EwPymDytsHyzFp5YoL,Should I get 2 times one ETH?,10.096352,10.096352
1,🍯dao.eth,Should I get 2 times one ETH,QmQ1s8FHhX2FtFGRcfbbdJmT1sU8EwPymDytsHyzFp5YoL,Should I get 2 times one ETH?,0.000000,10.096352
2,🍯dao.eth,What is the next achievement we should fund be...,QmZwnht7pRE2xGnUoQMU37eoNV8eEnobrejPZwDzdpBjaY,,0.498913,0.498913
3,🍯dao.eth,What is the next achievement we should fund be...,QmZwnht7pRE2xGnUoQMU37eoNV8eEnobrejPZwDzdpBjaY,,0.000000,0.498913
4,🍯dao.eth,Should the 🍯️DAO set its Gnosis Safe as the co...,QmWwo3TkBtUybd7wcwix6yuGtApR427FBCxiSGJNwowDoN,This is a proposal to set the controller of th...,2.711773,2.711773
5,🍯dao.eth,Should the 🍯️DAO set its Gnosis Safe as the co...,QmWwo3TkBtUybd7wcwix6yuGtApR427FBCxiSGJNwowDoN,This is a proposal to set the controller of th...,0.000000,2.711773
6,🍯dao.eth,Test,QmZQkuw2DFuut5FdYrDji2fKoHW3rWbQd9TbcWgDSufoHa,Test,2.275148,5.901114
7,🍯dao.eth,Test,QmZQkuw2DFuut5FdYrDji2fKoHW3rWbQd9TbcWgDSufoHa,Test,3.625966,5.901114
8,🍯dao.eth,Donate 1 BAL on Gitcoin,QmV7WfE1vxPdwNYoom57zTRBVB38hWRPCZJgXEgD2zxK65,Test a donation of 1Bal to the reality.eth Git...,2.476520,2.476520
9,🍯dao.eth,Donate 1 BAL on Gitcoin,QmV7WfE1vxPdwNYoom57zTRBVB38hWRPCZJgXEgD2zxK65,Test a donation of 1Bal to the reality.eth Git...,0.000000,2.476520


In [83]:
space_name = spaces_list.iat[2,0]

proposals2 = snapshot.Query.proposals(
  orderBy='created',
  orderDirection='desc',
  first=1000,
  where=[
    snapshot.Proposal.space == space_name,
    snapshot.Proposal.state == 'closed'
    ##snapshot.Proposal.title == 'OIP-18: Reward rate framework and reduction',
  ]
)

proposals_snapshots2 = sg.query_df([
    proposals2.title,
    proposals2.id,
    proposals2.body,
    proposals2.scores,
    proposals2.scores_total
])

proposals_snapshots2.insert(0,'dao',space_name)

proposals_snapshots2

,dao,proposals_title,proposals_id,proposals_body,proposals_scores,proposals_scores_total
0,허경영.eth,why 5,0xc340fabba20d23630113a5783957ee786954ccd968e1...,wht,0,0
1,허경영.eth,why 5,0xc340fabba20d23630113a5783957ee786954ccd968e1...,wht,0,0
2,허경영.eth,why 5,0xfd06a5d0eb8d2b50ca98280cebd30785b4435def91c6...,wht,7,7
3,허경영.eth,why 5,0xfd06a5d0eb8d2b50ca98280cebd30785b4435def91c6...,wht,0,7
4,허경영.eth,why,0x50f96ae5e39556ff8acd4eb9dc42c61d438f31caa221...,wht,0,0
5,허경영.eth,why,0x50f96ae5e39556ff8acd4eb9dc42c61d438f31caa221...,wht,0,0
6,허경영.eth,hi5,0xada54df6c07216c894e7df55969799054334e32c997b...,hi5,7,7
7,허경영.eth,hi5,0xada54df6c07216c894e7df55969799054334e32c997b...,hi5,0,7
8,허경영.eth,hi4,0x78a6dae6010a48caca5f01ecf18c2e642cb6c9f16439...,hi4,0,0
9,허경영.eth,hi4,0x78a6dae6010a48caca5f01ecf18c2e642cb6c9f16439...,hi4,0,0


In [84]:
proposals3 = []
t=3
while t<=50: ##numspaces when out of testing
  space_name = spaces_list.iat[t,0]
  proposals3 = snapshot.Query.proposals(
    orderBy='created',
    orderDirection='desc',
    first=1000,
    skip = 1000*t,
    where=[
      snapshot.Proposal.space == space_name,
      snapshot.Proposal.state == 'closed'
      ]
  )
  proposals_snapshots3 = sg.query_df([
    proposals3.title,
    proposals3.id,
    proposals3.body,
    proposals3.scores,
    proposals3.scores_total
])
  proposals_snapshots3.insert(0,'dao',space_name)
  proposals_snapshots = pd.concat([proposals_snapshots,proposals_snapshots2,proposals_snapshots3])
  t=t+1
  l=len(proposals_snapshots3)
  if t%10 == 0: print (t,space_name,l)
  if len(proposals_snapshots3)>0:
    print(t,space_name,l)

proposals_snapshots

10 狂奔的蜗牛5.eth 0
20 深圳刘先生.eth 0
30 掌上中药网平台.eth 0
40 商丘工学院.eth 0
50 中华人民共和国中央人民政府.eth 0
60 zzghcm001.eth 0
70 zwei02.eth 0
80 zorromiaotommy.eth 0
90 zonas.eth 0
100 zodiac-dao.eth 0


,dao,proposals_title,proposals_id,proposals_body,proposals_scores,proposals_scores_total
0,🍯dao.eth,Should I get 2 times one ETH,QmQ1s8FHhX2FtFGRcfbbdJmT1sU8EwPymDytsHyzFp5YoL,Should I get 2 times one ETH?,10.096352,10.096352
1,🍯dao.eth,Should I get 2 times one ETH,QmQ1s8FHhX2FtFGRcfbbdJmT1sU8EwPymDytsHyzFp5YoL,Should I get 2 times one ETH?,0.000000,10.096352
2,🍯dao.eth,What is the next achievement we should fund be...,QmZwnht7pRE2xGnUoQMU37eoNV8eEnobrejPZwDzdpBjaY,,0.498913,0.498913
3,🍯dao.eth,What is the next achievement we should fund be...,QmZwnht7pRE2xGnUoQMU37eoNV8eEnobrejPZwDzdpBjaY,,0.000000,0.498913
4,🍯dao.eth,Should the 🍯️DAO set its Gnosis Safe as the co...,QmWwo3TkBtUybd7wcwix6yuGtApR427FBCxiSGJNwowDoN,This is a proposal to set the controller of th...,2.711773,2.711773
...,...,...,...,...,...,...
39,허경영.eth,test3,0x6a17f472c1f1679e64ace47ef5ef6cc9d13f91c8886a...,test3,0.000000,11.000000
40,허경영.eth,test,0x077dfd4c9e16905837e54b3939f5543bc1414cbfbe41...,test,0.000000,7.000000
41,허경영.eth,test,0x077dfd4c9e16905837e54b3939f5543bc1414cbfbe41...,test,7.000000,7.000000
42,허경영.eth,test,0x4675e59a92310e1523690b07e67e7febb9d3e96c4bc4...,test,7.000000,7.000000


In [85]:
len(proposals_snapshots)

4322

We are going to use this to timestamp the filenames

In [87]:
snapshot.Proposal.datetime = SyntheticField(
  lambda timestamp: str(datetime.fromtimestamp(timestamp)),
  SyntheticField.STRING,
  snapshot.Proposal.end,
  )


In [90]:
##spit out the file!
file = input('Selet a folder') ##enter your file path here - the file is in the repo "summary_stats.csv".
path =file+'/votes_table_'+str(date.today().strftime("%b-%d-%Y"))+'_'+str(len(proposals_snapshots))+'_snapshots.csv'
proposals_snapshots.to_csv(path, index = False)
print('file saved')

file saved


Let's pull the choices each voter made. As always, I prime the pump with a single pull, then append more records/

In [91]:
proposal_id = proposals_snapshots.iloc[1,2]
voting_snapshots_list = sg.query_df([
  vote_tracker.id,
  vote_tracker.voter,
  vote_tracker.created,
  vote_tracker.choice,
  vote_tracker.vp
])
voting_snapshots_list

""


In [92]:
voting_snapshots_list['Proposal'] = proposal_id
print(voting_snapshots_list)

Empty DataFrame
Columns: [Proposal]
Index: []


In [95]:
n=2
total_props = (len(proposals_snapshots))-1
while n<=50: ##total_props:
  proposal_id = proposals_snapshots.iloc[n,2]
  if n%10 == 0: print (n,proposal_id)
  n=n+1
  vote_tracker = snapshot.Query.votes(
  orderBy = 'created',
  orderDirection='desc',
  first=10000,
  where=[
    snapshot.Vote.proposal == proposal_id
  ])
  voting_snapshots_list2 = sg.query_df([
    vote_tracker.id,
    vote_tracker.voter,
    vote_tracker.created,
    vote_tracker.choice,
    vote_tracker.vp
  ])
  voting_snapshots_list2['Proposal'] = proposal_id
  voting_snapshots_list = pd.concat([voting_snapshots_list,voting_snapshots_list2])

10 0xc340fabba20d23630113a5783957ee786954ccd968e18cd4cf2d57874f273973
20 0x5084fa5a12140a987cafe66558e29a0b4923d234978f5b183c39a9ee07ccfc94
30 0xc8edaa15b143bc113a9f6a383f69d6937f57a02cd04b3427d4c2388796a84c09
40 0x18ed8c70621f6740c1f9775a55070a6346e5c991884deef75f49c614b62cc84d
50 0x077dfd4c9e16905837e54b3939f5543bc1414cbfbe4175f9f988b364e86256c7


In [96]:
voting_snapshots_list

,Proposal,votes_id,votes_voter,votes_created,votes_choice,votes_vp
0,QmZwnht7pRE2xGnUoQMU37eoNV8eEnobrejPZwDzdpBjaY,QmesJhaqKBtLK7zpC9yCZrLMTqgJwFMkBtLkJg18u51j6k,0xdde553b4fC83EaA9802EcF7Fe35F7701bfc7cE56,1.627554e+09,1.0,0.498913
0,QmZwnht7pRE2xGnUoQMU37eoNV8eEnobrejPZwDzdpBjaY,QmesJhaqKBtLK7zpC9yCZrLMTqgJwFMkBtLkJg18u51j6k,0xdde553b4fC83EaA9802EcF7Fe35F7701bfc7cE56,1.627554e+09,1.0,0.498913
0,QmWwo3TkBtUybd7wcwix6yuGtApR427FBCxiSGJNwowDoN,QmcZ8TpRkGpjfNDgta7YND3HBHbVJ4TwJN42kANstSQiKQ,0x7B2e78D4dFaABA045A167a70dA285E30E8FcA196,1.618525e+09,1.0,2.061410
1,QmWwo3TkBtUybd7wcwix6yuGtApR427FBCxiSGJNwowDoN,QmUqNwHL9aFG1tgdkBytg3X9JTRLnSoYLnDSdNGeQMUfcY,0xd714Dd60e22BbB1cbAFD0e40dE5Cfa7bBDD3F3C8,1.618496e+09,1.0,0.650362
0,QmWwo3TkBtUybd7wcwix6yuGtApR427FBCxiSGJNwowDoN,QmcZ8TpRkGpjfNDgta7YND3HBHbVJ4TwJN42kANstSQiKQ,0x7B2e78D4dFaABA045A167a70dA285E30E8FcA196,1.618525e+09,1.0,2.061410
...,...,...,...,...,...,...
0,0x6a17f472c1f1679e64ace47ef5ef6cc9d13f91c8886a...,0xfd19773e0f156e3ba597324cf1d6cf14d32ef63f2b74...,0x5cF6e510E0C123E1e839376F8Ea1900Bb9B9A6C1,1.650735e+09,1,7.000000
1,0x6a17f472c1f1679e64ace47ef5ef6cc9d13f91c8886a...,0x66a006d6e1e4541879af35f5b0177179356f45f2ff11...,0x8A39eC10AE914CD71DE75A766D7Beb78C1286D0e,1.650735e+09,1,4.000000
0,0x6a17f472c1f1679e64ace47ef5ef6cc9d13f91c8886a...,0xfd19773e0f156e3ba597324cf1d6cf14d32ef63f2b74...,0x5cF6e510E0C123E1e839376F8Ea1900Bb9B9A6C1,1.650735e+09,1,7.000000
1,0x6a17f472c1f1679e64ace47ef5ef6cc9d13f91c8886a...,0x66a006d6e1e4541879af35f5b0177179356f45f2ff11...,0x8A39eC10AE914CD71DE75A766D7Beb78C1286D0e,1.650735e+09,1,4.000000


In [97]:
##spit out the file!
file = input('Selet a folder') ##enter your file path here - the file is in the repo "summary_stats.csv".
path =file+'/votes_table_'+str(date.today().strftime("%b-%d-%Y"))+'_'+str(len(voting_snapshots_list))+'_proposals.csv'
voting_snapshots_list.to_csv(path, index = False)
print('file saved')

And finally, the body of each election's choices. Not critical but useful to have.

In [98]:
proposals_choices = sg.query(proposals.choices)
proposals_choices

TypeError: 'FieldPath' object is not iterable

In [76]:
proposals_choices = sg.query(proposals.choices)

TypeError: 'FieldPath' object is not iterable

In [38]:
##placing 25 options just in case. Could be more, but I hope not
proposals_choices = pd.DataFrame(proposals_choices, columns = ['option_1', 'option_2', 'option_3', 'option_4', 'option_5','option_6','option_7','option_8','option_9','option_10','option_11','option_12','option_13','option_14','option_15','option_16','option_17','option_18','option_19','option_20','option_21','option_22','option_24','option_24','option_25]']

SyntaxError: '(' was never closed (1609945154.py, line 2)

Let's pull the choices each voter made. As always, I prime the pump with a single pull, then append more records/

In [61]:
proposal_id = proposals_snapshots.iloc[1,2]
voting_snapshots_list = sg.query_df([
  vote_tracker.id,
  vote_tracker.voter,
  vote_tracker.created,
  vote_tracker.choice,
  vote_tracker.vp
])
voting_snapshots_list

,votes_id,votes_voter,votes_created,votes_choice,votes_vp
0,0x6df5f88dbe7c5c114cb9991fd93a2395c669e1608ca6...,0x5cF6e510E0C123E1e839376F8Ea1900Bb9B9A6C1,1654154075,1,7


In [62]:
voting_snapshots_list['Proposal'] = proposal_id
print(voting_snapshots_list)

                                            votes_id  \
0  0x6df5f88dbe7c5c114cb9991fd93a2395c669e1608ca6...   

                                  votes_voter  votes_created  votes_choice  \
0  0x5cF6e510E0C123E1e839376F8Ea1900Bb9B9A6C1     1654154075             1   

   votes_vp                                        Proposal  
0         7  QmZwnht7pRE2xGnUoQMU37eoNV8eEnobrejPZwDzdpBjaY  


In [63]:
n=2
total_props = (len(proposals_snapshots))-1
while n<=50: ##total_props:
  proposal_id = proposals_snapshots.iloc[n,2]
  if n%10 == 0: print (n,proposal_id)
  n=n+1
  vote_tracker = snapshot.Query.votes(
  orderBy = 'created',
  orderDirection='desc',
  first=10000,
  where=[
    snapshot.Vote.proposal == proposal_id
  ])
  voting_snapshots_list2 = sg.query_df([
    vote_tracker.id,
    vote_tracker.voter,
    vote_tracker.created,
    vote_tracker.choice,
    vote_tracker.vp
  ])
  voting_snapshots_list2['Proposal'] = proposal_id
  voting_snapshots_list = pd.concat([voting_snapshots_list,voting_snapshots_list2])

10 0x5084fa5a12140a987cafe66558e29a0b4923d234978f5b183c39a9ee07ccfc94
20 0x18ed8c70621f6740c1f9775a55070a6346e5c991884deef75f49c614b62cc84d
30 0xada54df6c07216c894e7df55969799054334e32c997b7b9565be62084f0ad366
40 0xd3025ca0b81f6a390f0955d109f4d1cae3df44d7a9eb956bbbc6cd7ce80891d0
50 0xfd06a5d0eb8d2b50ca98280cebd30785b4435def91c696c86ef6d987095cf918


In [64]:
voting_snapshots_list

,votes_id,votes_voter,votes_created,votes_choice,votes_vp,Proposal
0,0x6df5f88dbe7c5c114cb9991fd93a2395c669e1608ca6...,0x5cF6e510E0C123E1e839376F8Ea1900Bb9B9A6C1,1.654154e+09,1.0,7.000000,QmZwnht7pRE2xGnUoQMU37eoNV8eEnobrejPZwDzdpBjaY
0,QmcZ8TpRkGpjfNDgta7YND3HBHbVJ4TwJN42kANstSQiKQ,0x7B2e78D4dFaABA045A167a70dA285E30E8FcA196,1.618525e+09,1.0,2.061410,QmWwo3TkBtUybd7wcwix6yuGtApR427FBCxiSGJNwowDoN
1,QmUqNwHL9aFG1tgdkBytg3X9JTRLnSoYLnDSdNGeQMUfcY,0xd714Dd60e22BbB1cbAFD0e40dE5Cfa7bBDD3F3C8,1.618496e+09,1.0,0.650362,QmWwo3TkBtUybd7wcwix6yuGtApR427FBCxiSGJNwowDoN
0,QmQHbreEmbhDsDrRDP8cV5r8Lim3m2iu7iUNT6wjAWR1V1,0x7B2e78D4dFaABA045A167a70dA285E30E8FcA196,1.616704e+09,2.0,2.414856,QmZQkuw2DFuut5FdYrDji2fKoHW3rWbQd9TbcWgDSufoHa
1,QmV8Dg5jyYJfv2RuqW6WwMX5BC1MmWTUWnf6Lh4e2bdihj,0xd714Dd60e22BbB1cbAFD0e40dE5Cfa7bBDD3F3C8,1.616685e+09,2.0,0.200417,QmZQkuw2DFuut5FdYrDji2fKoHW3rWbQd9TbcWgDSufoHa
2,QmXxQA5fLUnxQbvCaQX1F7RYp12BpkP8MgKPoK2GdYteqU,0x08Fb5797b3D751D022736F6DF6006a39c049c2C6,1.616337e+09,1.0,0.159626,QmZQkuw2DFuut5FdYrDji2fKoHW3rWbQd9TbcWgDSufoHa
3,QmYGGETLKWCAEs3U63H77n2MKyxM2r3xMmaSRzQa9yKhxS,0x44Ba78D48D0981c765DEE84C048Cea662eAe2E8f,1.616337e+09,1.0,0.196898,QmZQkuw2DFuut5FdYrDji2fKoHW3rWbQd9TbcWgDSufoHa
4,QmdCaXncTR5XS64BdF3U5av4cQSdc9ikUPcBKAzghXYMDE,0xE8Ef67a6677ba943c33b6C926598072e919b0B6D,1.616103e+09,2.0,1.010692,QmZQkuw2DFuut5FdYrDji2fKoHW3rWbQd9TbcWgDSufoHa
5,QmezEYaRALq6krgD6thubabxdrceTrRkM2xEFpJDMFzvkp,0x24F15402C6Bb870554489b2fd2049A85d75B982f,1.616035e+09,1.0,1.918624,QmZQkuw2DFuut5FdYrDji2fKoHW3rWbQd9TbcWgDSufoHa
0,QmSNHx5ouhMNXn87BXqK5HGQdQzbTTLBjfoqU4Z9YTNCwF,0x7B2e78D4dFaABA045A167a70dA285E30E8FcA196,1.616023e+09,1.0,2.476520,QmV7WfE1vxPdwNYoom57zTRBVB38hWRPCZJgXEgD2zxK65


In [60]:
##spit out the file!
file = input('Selet a folder') ##enter your file path here - the file is in the repo "summary_stats.csv".
path = '/Users/ianbrillembourg/PycharmProjects/snapshot_governance/votes_table_'+str(date.today().strftime("%b-%d-%Y"))+'_'+str(len(voting_snapshots_list))+'_proposals.csv'
voting_snapshots_list.to_csv(path, index = False)

And finally, the body of each election's choices. Not critical but useful to have.

In [ ]:
voting_snapshots_list['Proposal'] = proposal_id
print(voting_snapshots_list)

In [ ]:
proposals_choices = sg.query(proposals.choices)
for x<=total_props:
  voting_snapshots_list['Proposal'] = proposal_id
proposals_choices

NameError: name 'proposals_choices' is not defined

In [32]:
##placing 25 options just in case. Could be more, but I hope not
proposals_choices = pd.DataFrame(proposals_choices, columns = ['option_1', 'option_2', 'option_3', 'option_4', 'option_5','option_6','option_7','option_8','option_9','option_10','option_11','option_12','option_13','option_14','option_15','option_16','option_17','option_18','option_19','option_20','option_21','option_22','option_24','option_24','option_25]']

SyntaxError: incomplete input (1609945154.py, line 2)

In [ ]:
dao_governance_view = pd.concat([proposals_snapshots,proposals_choices], axis=1)

In [ ]:
dao_governance_view